![Cardiotoxicity](cardiotoxicityProjectTitle.jpg)
#MALDI Parser
![maldi plate](maldi_anchor_s.jpg)

#Import Modules


In [ ]:
import xlrd
import pandas as pd
import numpy as np
import re
import warnings
import sys
import time
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

#Functions
- Finding missed peak(s) 

In [ ]:
#Finding Missed Peak
def detect_missed(mzs):
    mzs_missing=[]
    
    for normal_mz in normal_mzs:
        for mz in mzs:
            if mz<normal_mz+6 and mz>normal_mz-6:   #Searching in this range
                break
        else:
            mzs_missing.append(normal_mz)
    return mzs_missing

- Finding redundant peak(s)

In [ ]:
#Finding Redundant Peak
def detect_redundant(mzs):
    mzs_adjacents=[]    

    for normal_mz in normal_mzs:
        d=9.99
        m=0
        for mz in mzs:
            if abs(normal_mz-mz)<d:
                d=abs(normal_mz-mz)
                m=mz
        mzs_adjacents.append(m)
   
    return mzs_adjacents

- Display the layout

In [ ]:
#Layout Display
def layout(names):
    layout_names=[]
    layout_series=[]
    for name in names:
        patt4=r'[\D\d]+_0_'
        sample_name=re.findall(patt4, name)[0].replace('_0_', '').replace('-', '_')
        
        if sample_name.find('standard')!=-1 or sample_name.find('Standard')!=-1:
            sample_name=sample_name.replace('standard', '').replace('Standard', '').replace('_', '')       

        patt5=r'_0_[\D\d]+_1$'
        id_name=re.findall(patt5, name)[0].replace('_0_', '').replace('_1', '')

        if len(id_name)==2:
            id_name=id_name[0]+'0'+id_name[1]

         
        layout_names.append((id_name, sample_name))

    d=96-len(layout_names)

    for i in range(d):
        layout_names.append(('Z'+str(i),np.nan))

    sample_dict=dict(layout_names)
    s=pd.Series(sample_dict).sort_index()
    df=pd.DataFrame(np.array(s).reshape(8,12),
                           columns=[1,2,3,4,5,6,7,8,9,10,11,12],
                           index=['A','B','C','D','E','F','G','H'])
    return df

- Change certain value into NA

In [ ]:
#Make certain cells blank (NaN)
def blank():
    df['SAA std'][df['level_1']==1]=np.nan
    df['SAA std'][df['level_1']==2]=np.nan
    df['SAA std'][df['level_1']==3]=np.nan

    df['SAA avg'][df['level_1']==1]=np.nan
    df['SAA avg'][df['level_1']==2]=np.nan
    df['SAA avg'][df['level_1']==3]=np.nan

    df['hep std'][df['level_1']==1]=np.nan
    df['hep std'][df['level_1']==2]=np.nan
    df['hep std'][df['level_1']==3]=np.nan

    df['hep avg'][df['level_1']==1]=np.nan
    df['hep avg'][df['level_1']==2]=np.nan
    df['hep avg'][df['level_1']==3]=np.nan

    df['SAA/IS_SAA'][df['level_1']==1]=np.nan
    df['SAA/IS_SAA'][df['level_1']==2]=np.nan
    df['SAA/IS_SAA'][df['level_1']==3]=np.nan

    df['hep/hhep'][df['level_1']==1]=np.nan
    df['hep/hhep'][df['level_1']==2]=np.nan
    df['hep/hhep'][df['level_1']==3]=np.nan

    df['SAA CV'][df['level_1']==1]=np.nan
    df['SAA CV'][df['level_1']==2]=np.nan
    df['SAA CV'][df['level_1']==3]=np.nan

    df['hep CV'][df['level_1']==1]=np.nan
    df['hep CV'][df['level_1']==2]=np.nan
    df['hep CV'][df['level_1']==3]=np.nan

    df['name'][df['level_1']==1]=np.nan
    df['name'][df['level_1']==2]=np.nan
    df['name'][df['level_1']==3]=np.nan
        
    df['hep avg'][df['Tail']%2==0]=np.nan
    df['SAA avg'][df['Tail']%2==0]=np.nan
    df['hep std'][df['Tail']%2==0]=np.nan
    df['SAA std'][df['Tail']%2==0]=np.nan
    df['hep CV'][df['Tail']%2==0]=np.nan
    df['SAA CV'][df['Tail']%2==0]=np.nan

    df['tag'][df['level_1']==1]=np.nan
    df['tag'][df['level_1']==2]=np.nan
    df['tag'][df['level_1']==3]=np.nan
    df['tag'][df['Tail']%2==0]=np.nan

#Main Program Flow
We chose four peaks for detection: **1526, 1536** (Hepcidin), **2791, 2819** (SAA)

In [ ]:
#////////////////MAIN///////////////////////////////////////////////
#Ignoring Warnings
warnings.filterwarnings("ignore")

#Default Peaks
normal_mzs=[1526, 1536, 2791, 2819]
normal_rows=len(normal_mzs)

#Import File
filename='Raw MALDI.xlsx'
book=xlrd.open_workbook(filename)
print('Source file: '+sys.path[0]+filename+' loaded!')

#Extraction
nsheets=book.nsheets

sheet_names=book.sheet_names()
sheets={}

for sheet_name in sheet_names:    
    nrows=book.sheet_by_name(sheet_name).nrows
    current_header=book.sheet_by_name(sheet_name).row_values(2) 
    current_data=[book.sheet_by_name(sheet_name).row_values(i) for i in range(3, nrows)]
    sheets[sheet_name]=pd.DataFrame(current_data, columns=current_header)   #DataFrame Construction
#Feedback
print('Data Extracted!')

#Dealing with the Missing Peak(s)

In [ ]:
#///////////////////Main Loop///////////
peak_missing_amount=0
peak_redundant_amount=0
peak_redundant_item=[]
peak_missing_item=[]
peak_repeated_amount=0
peak_repeated_item=[]

for sheet_name in sheet_names:
    df=sheets[sheet_name]

    actual_rows=len(df.index)
    mzs=list(df['m/z'])

    #////////////////////Unique!!!!///////////////////////////////////////////////
    for mz in mzs:
        if len(df[df['m/z']==mz].index)>1:
            del_index=list(df[df['m/z']==mz].index)
            del_index.pop(0)
            df=df.drop(del_index)

            mzs=list(df['m/z'])

            peak_repeated_amount+=1
            peak_repeated_item.append(sheet_name.replace('_0_', ' @ ').replace('_1', ''))
    #////////////////////////////////////////////////////////////////////////////

    df=df.sort_index(by='m/z')
    df=df.reset_index().drop('index', axis=1)

    actual_rows=len(df.index)

    #///////Larger than normal: Redundant///////////
    if actual_rows>normal_rows:
        mzs=list(df['m/z'])
        mzs_adjacents=detect_redundant(mzs)    #Call Function          
        df=pd.concat([df.ix[df[df['m/z']==mzs_adjacent].index] for mzs_adjacent in mzs_adjacents])        

        #////Memorize the Redundant One//////////////////    
        peak_redundant_amount+=1
        peak_redundant_item.append(sheet_name.replace('_0_', ' @ ').replace('_1', ''))

    #/////Sort & Reindex////////////
    df=df.sort_index(by='m/z')
    df=df.reset_index().drop('index', axis=1)

    actual_rows=len(df.index)
    
    ##///////Less than normal: Missing///////
    if actual_rows<normal_rows:
        mzs=list(df['m/z'])
        mzs_missing=detect_missed(mzs)  #Call Function 

        i=actual_rows
        
        for mz_missing in mzs_missing:
            df.ix[i]=0
            df['m/z'].ix[i]=mz_missing
            i+=1

        #////Memorize the Missing One///////
        peak_missing_amount+=1
        peak_missing_item.append(sheet_name.replace('_0_', ' @ ').replace('_1', ''))

    df=df.sort_index(by='m/z')
    df=df.reset_index().drop('index', axis=1)

    actual_rows=len(df.index)    

    #///////Again! Larger than normal: Redundant////////
    if actual_rows>normal_rows:
        mzs=list(df['m/z'])
        mzs_adjacents=detect_redundant(mzs)     #Call Function          
        df=pd.concat([df.ix[df[df['m/z']==mzs_adjacent].index] for mzs_adjacent in mzs_adjacents])       

        #/Memorize the Redundant One
        peak_redundant_amount+=1
        peak_redundant_item.append(sheet_name.replace('_0_', ' @ ').replace('_1', ''))

    #Sort & Reindex/////////////
    df=df.sort_index(by='m/z')
    df=df.reset_index().drop('index', axis=1)

    #Descriptive Calculation
    hep_ratio=df['Area'].ix[2]/df['Area'].ix[3]
    saa_ratio=df['Area'].ix[0]/df['Area'].ix[1]

    #Regular Expressions////Matching Sample Names
    patt=r'[\D\d]+_0_'
    name=re.findall(patt, sheet_name)[0].replace('_0_', '').replace('-', '_')

    #Regular Expressions////Matching Vial NOs
    patt1=r'_0_[\D\d]+_1$'
    id=re.findall(patt1, sheet_name)[0].replace('_0_', '').replace('_1', '')

    #Regular Expressions////Matching Tails of the Vials
    patt2=r'\d+$'
    tail=int(re.findall(patt2, id)[0])

    if len(id)==2:
        id=id[0]+'0'+id[1]

    df['Tail']=tail
    df['id']=id
    df['name']=name
    df['hep/hhep']=hep_ratio
    df['SAA/IS_SAA']=saa_ratio
    df['tag']=name

    sheets[sheet_name]=df
print('Done!')

#Combination & Unique

In [ ]:
#Combining DataFrame
#Feedback: Combining
print('Combining...')
df=pd.concat([sheets[sheet_name] for sheet_name in sheet_names], keys=sheet_names)

#Feedback: Start Parsing
print('Parsing...')

#去重
names=list(df['name'].unique())

#Description Statistics Initialization
df['hep avg']=0
df['hep std']=0
df['hep CV']=0

df['SAA avg']=0
df['SAA std']=0
df['SAA CV']=0

#///Change the sheetnames from indeces into column names
df=df.reset_index()
print('Done!')

#Descriptive Statistics for the same names

In [ ]:
#////////Loop of Processing Data in the Same Names///////////////
for name in names:
    avg=df['hep/hhep'][df['name']==name].mean()
    df['hep avg'][df['name']==name]=avg

    std=df['hep/hhep'][df['name']==name].std()
    df['hep std'][df['name']==name]=std

    cv=std*100/avg
    df['hep CV'][df['name']==name]=cv

    #////////////////////////////////////////////
    
    avg=df['SAA/IS_SAA'][df['name']==name].mean()
    df['SAA avg'][df['name']==name]=avg

    std=df['SAA/IS_SAA'][df['name']==name].std()
    df['SAA std'][df['name']==name]=std

    cv=std*100/avg
    df['SAA CV'][df['name']==name]=cv

    #/////////////////////////////////////////////

    if name.find('standard') != -1 or name.find('Standard') != -1:
        tag=name.replace('standard', '').replace('Standard', '').replace('_', '').replace(' ', '')
        df['tag'][df['name']==name]=tag        

#Export the excel file

In [ ]:
#Calling Function
print('Trimming...')
blank()

#Sorting
df=df.sort_index(by=['id', 'level_1'])
df=df.set_index('level_0')

#Output
print('Exporting...')
df.to_csv('.\\combined.csv')

dfl=layout(sheet_names)
dfl.to_csv('.\\layout.csv')
print('Files Exported!')

In [ ]:
dfl

In [ ]:
#Final Feedback
print('\nDone!\n')

print('Total Sample Amount: ', nsheets)

print('\tPeak Missing: ', peak_missing_amount)
print('\t\tMissing Item(s): ', peak_missing_item, '\n')

print('\tPeak Redundant: ', peak_redundant_amount)
print('\t\tRedundant Item(s): ', peak_redundant_item, '\n')

print('\tPeak Repeated: ', peak_repeated_amount)
print('\t\tRepeated Item(s): ', peak_repeated_item, '\n')

print('Combined Document Exported: ', sys.path[0]+'\\combined.csv')
print('Layout Document Exported: ', sys.path[0]+'\\layout.csv\n')

#Drawing Standard Curve
- Extraction of Standard Data

In [ ]:
standard_list=['250', '125', '62.5', '31.25', '15.6', '7.8', '3.9', '1.9', '0']
standard_value={}

for current in standard_list:
    standard_value[float(current)]=float(df[df['tag']==current]['hep avg'])
concentration=list(standard_value.keys())
intensity=list(standard_value.values())

regression_result=stats.linregress(concentration, intensity)
corr_result=stats.pearsonr(concentration, intensity)

x=range(-50, 300)
y=regression_result[0]*x+regression_result[1]

- Plotting Scatters and Line

In [ ]:
plt.scatter(concentration, intensity)
plt.plot(x, y, color='r')
plt.savefig('StandardCurve.png', dpi=400)

- Showing the Correlation Coefficient and P Value

In [ ]:
print('r=', corr_result[0])
print('P=', corr_result[1])